# Introduction to GraphQL

GraphQL is a query language for APIs and a runtime for executing those queries against your data. It provides a more complete and understandable description of the data in your API and gives clients the power to ask for exactly what they need and nothing more. Developed by Facebook in 2012 and open-sourced in 2015, GraphQL supports reading, writing, and subscribing to changes to data.


## Key Features

### 1. Precise Queries

With GraphQL, you can request only the data you need, avoiding over-fetching or under-fetching of data, a common issue encountered with REST APIs.

### 2. Single Endpoint
Unlike REST, where each resource has a separate endpoint, GraphQL APIs expose a single endpoint for all interactions, streamlining the communication process and making API management easier.


Below is a Python script using the requests library to query the GitHub GraphQL API. This script will fetch details of a GitHub repository. Before running the script, make sure to replace YOUR_GITHUB_TOKEN with your actual GitHub token.

In [3]:
import requests
import json

# Define the GraphQL query
query = """
{
  repository(owner:"octocat", name:"Hello-World") {
    name
    description
    forkCount
    stargazers {
      totalCount
    }
  }
}
"""

# Define the headers, including your GitHub token
headers = {
    "Authorization": "Bearer YOUR_GITHUB_TOKEN",
    "Content-Type": "application/json"
}

# Make the API request
response = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)

# Parse the response
data = response.json()

# Print the data
print(json.dumps(data, indent=4))

{
    "data": {
        "repository": {
            "name": "Hello-World",
            "description": "My first repository on GitHub!",
            "forkCount": 2315,
            "stargazers": {
                "totalCount": 2454
            }
        }
    }
}


<br>
Here is a comparison with a REST API query is a great idea to emphasize the benefits of GraphQL. Here's how you could fetch the same information from the GitHub REST API using Python:
<br><br>

In [5]:
import requests

# Define the REST API endpoint
endpoint = "https://api.github.com/repos/octocat/Hello-World"

# Define the headers, including your GitHub token
headers = {
    "Authorization": "Bearer YOUR_GITHUB_TOKEN",
    "Content-Type": "application/json"
}

# Make the API request
response = requests.get(endpoint, headers=headers)

# Parse the response
data = response.json()

# Extract specific data fields
repo_name = data.get('name')
description = data.get('description')
fork_count = data.get('forks_count')
stargazers_count = data.get('stargazers_count')

# Create a dictionary to hold the extracted data
extracted_data = {
    "name": repo_name,
    "description": description,
    "forkCount": fork_count,
    "stargazers": {
        "totalCount": stargazers_count
    }
}

# Print the data
print(json.dumps(extracted_data, indent=4))

{
    "name": "Hello-World",
    "description": "My first repository on GitHub!",
    "forkCount": 2315,
    "stargazers": {
        "totalCount": 2454
    }
}


Now, let's compare the two approaches:

##### REST API
<b>Endpoint:</b> Each resource (like repository, user info, etc.) has a separate endpoint.
<b>Over-fetching/Under-fetching:</b> You receive all the data that the endpoint returns, which might be more or less than what you actually need.
<b>Flexibility:</b> Lesser flexibility as the structure of the response is predetermined by the server.


##### GraphQL
<b>Single Endpoint:</b> All queries are sent to a single endpoint.
<b>Precise Data Retrieval:</b> You can specify exactly which fields you want in the response, avoiding over-fetching or under-fetching of data.
<b>Flexibility:</b> Greater flexibility as you can structure the query to receive data in the format you prefer.
<br><br>

### 3. Real-Time Data with Subscriptions
GraphQL offers subscriptions that allow clients to receive real-time updates when data changes, providing a dynamic and responsive API interaction experience.

GraphQL Subscriptions are a way to push data from the server to the clients that choose to listen to real-time messages from the server. Subscriptions are built on top of the WebSocket protocol, which allows for continuous two-way communication between the client and server.

To showcase this example we will set about creating a local graphQL server using Apollo-server


#### Step 1: Setting Up Docker
Before we dive into creating our GraphQL server, we first need to set up a Docker environment. Here's how you can do it:

1. <b>Install Docker:</b> If not already installed, download and install Docker on your system.

2. <b>Create a Dockerfile:</b> In your project directory, create a file named Dockerfile with the following content to set up a Node.js environment:


<pre><code>
# Use the official Node.js image
FROM node:14

# Create a directory for the app
WORKDIR /usr/src/app

# Copy package.json and package-lock.json
COPY package*.json ./

# Install dependencies
RUN npm install

# Copy the local files to the container
COPY . .

# Expose the port the app runs on
EXPOSE 4000

# Define the command to run the app
CMD [ "node", "server.js" ]
</code></pre>


3. <b>Create a .dockerignore File:</b> Create a .dockerignore file in your project directory with the following content to prevent node_modules and other unnecessary files from being copied to the Docker image:

<pre><code>node_modules
npm-debug.log
</code></pre>


#### Step 2: Setting Up a Local GraphQL Server
Now, we will proceed to set up our GraphQL server inside a Docker container.

1. <b>Create a New Node.js Project:</b> In your project directory, create a `package.json` file with basic details about the project. You can generate this file by running `npm init -y` in a Node.js environment and then copying it to your project directory.

2. <b>Install Necessary Packages:</b> Specify the necessary packages (apollo-server and graphql) in the dependencies section of the package.json file:


<pre><code>
"dependencies": {
  "apollo-server": "^3.5.0",
  "graphql": "^15.8.0",
  "graphql-subscriptions": "^2.0.0"
}
</code></pre>


3. <b>Create a GraphQL Schema and Resolvers:</b> Create a file named `schema.js` in your project directory and add the following content:

<pre><code>
<font color="blue">const</font> { gql } = require(<font color="green">'apollo-server'</font>);

<font color="blue">const</font> typeDefs = gql<font color="green">`
  type Message {
    id: ID!
    user: String!
    content: String!
  }

  type Subscription {
    messageAdded: Message
  }

  type Query {
    messages: [Message]
  }

  type Mutation {
    addMessage(user: String!, content: String!): Message
  }
`</font>;

<font color="magenta">module</font>.exports = typeDefs;
</code></pre>

In this schema:

- We define a Message type with fields id, user, and content.
- We define a Subscription type with a messageAdded field that returns a Message type. This will be used to push real-time updates to subscribers.
- We define a Query type with a messages field to fetch all messages.
- We define a Mutation type with an addMessage field to add new messages.

Next, create a file named `resolvers.js` in your project directory and add the following content:

<pre><code>
const { pubsub } = require('./server');
let messages = [];
let idCount = 0;

const resolvers = {
  Mutation: {
    addMessage: (_, { user, content }, { pubsub }) => {
      const message = { id: idCount++, user, content };
      messages.push(message);
      pubsub.publish('MESSAGE_ADDED', { messageAdded: message });
      return message;
    },
  },
  Subscription: {
    messageAdded: {
      subscribe: (_, __, { pubsub }) => pubsub.asyncIterator(['MESSAGE_ADDED']),
    },
  },
};

module.exports = resolvers;
</code></pre>


In this file:

- We define a messages array to hold message objects and an idCount variable to assign unique IDs to messages.
- We define resolvers for the messages query and the addMessage mutation.
- We define a resolver for the messageAdded subscription to handle real-time updates.


#### Step 3. Setting Up the Server

Next, we will create the server file where we will set up Apollo Server and initialize it with the schema and resolvers we created in the previous step.

1. <b>Create a Server File:</b> Create a file named `server.js` in your project directory and add the following content:

<pre><code>
const { ApolloServer } = require('apollo-server');
const { PubSub } = require('graphql-subscriptions');
const typeDefs = require('./schema');
const pubsub = new PubSub();
const resolvers = require('./resolvers');

const server = new ApolloServer({ typeDefs, resolvers, context: { pubsub } });

server.listen().then(({ url, subscriptionsUrl }) => {
  console.log(`🚀 Server ready at ${url}`);
  console.log(`🚀 Subscriptions ready at ${subscriptionsUrl}`);
});
</code></pre>


#### Step 4: Building and Running the Docker Container
Now that our GraphQL server setup is complete, we will build and run our Docker container.

1. <b>Build the Docker Image:</b> In your terminal, navigate to your project directory and run the command docker build -t graphql-server . to build the Docker image.

2. <b>Run the Docker Container:</b> After the build completes, run the command docker run -p 4000:4000 graphql-server to start the container. The GraphQL server should now be running at `http://localhost:4000`.


#### Step 5: Creating a Python Script to Interact with the Server

In [6]:
import sys
!{sys.executable} -m pip install asyncio websockets gql

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 6.2 MB/s eta 0:00:00
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config fi

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

def add_message(user, content):
    transport = RequestsHTTPTransport(url='http://localhost:4000/graphql')
    
    client = Client(
        transport=transport,
        fetch_schema_from_transport=True,
    )
    
    add_message_mutation = gql(
        """
        mutation addMessage($user: String!, $content: String!) {
            addMessage(user: $user, content: $content) {
                id
                user
                content
            }
        }
        """
    )

    params = {
        "user": user,
        "content": content
    }

    result = client.execute(add_message_mutation, variable_values=params)
    print(result)

# Example usage:
add_message("Alice", "Hello, world!")

<br><br>
In this next example we will see how to transition from messages to transactions to showcase the Subscriptions features of GraphQL

Make the below changes to `resolvers.ks`

<pre><code>
const { getUserById } = require('./data');
const { pubsub } = require('./pubsub');

let transactions = [];
let idCount = 0;

const resolvers = {
  Query: {
    transactions: () => transactions,
  },
  Mutation: {
    addTransaction: (_, { userId, amount }) => {
      const user = getUserById(userId);
      const transaction = { id: idCount++, userId, amount, user };
      transactions.push(transaction);
      pubsub.publish('TRANSACTION_ADDED', { transactionAdded: transaction });
      return transaction;
    },
  },
  Subscription: {
    transactionAdded: {
      subscribe: () => pubsub.asyncIterator(['TRANSACTION_ADDED']),
    },
  },
};

module.exports = resolvers;
</code></pre>

and the below changes to server.js

<pre><code>
const express = require('express');
const { execute, subscribe } = require('graphql');
const { SubscriptionServer } = require('subscriptions-transport-ws');
const { makeExecutableSchema } = require('@graphql-tools/schema');
const { graphqlHTTP } = require('express-graphql');
const { PubSub } = require('graphql-subscriptions');
const typeDefs = require('./schema');
const resolvers = require('./resolvers');
const http = require('http');

const pubsub = new PubSub();
const schema = makeExecutableSchema({ typeDefs, resolvers });
const app = express();

app.use('/graphql', graphqlHTTP({
  schema,
  graphiql: {
    subscriptionEndpoint: 'ws://localhost:4000/graphql'
  }
}));

const httpServer = http.createServer(app);

SubscriptionServer.create(
  {
    execute,
    subscribe,
    schema
  },
  {
    server: httpServer,
    path: '/graphql'
  }
);

httpServer.listen(4000, () => {
  console.log(`🚀 Server ready at http://localhost:4000/graphql`);
});
</code></pre>


#### Reason for changes

1. Introduction of Express: We introduced Express.js to have better control over the HTTP server and to integrate the GraphQL server more smoothly with the WebSocket server for subscriptions.

2. Using SubscriptionServer: We started using SubscriptionServer from the subscriptions-transport-ws package to create a WebSocket server for handling GraphQL subscriptions.

3. Schema Creation: We started using makeExecutableSchema to create the GraphQL schema manually, which is then used in both the GraphQL HTTP middleware and the WebSocket server.

Update the dependencies on package.json as below

<pre><code>
{
  "dependencies": {
    "express": "^4.17.3",
    "graphql": "^15.8.0",
    "graphql-subscriptions": "^1.2.1",
    "subscriptions-transport-ws": "^0.11.0",
    "express-graphql": "^0.12.0",
    "@graphql-tools/schema": "^8.1.0"
  }
}
</code></pre>

In [42]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

def add_transaction(user_id, amount):
    transport = RequestsHTTPTransport(url='http://localhost:4000/graphql')
    
    client = Client(
        transport=transport,
        fetch_schema_from_transport=True,
    )
    
    add_transaction_mutation = gql(
        """
        mutation addTransaction($userId: ID!, $amount: Float!) {
            addTransaction(userId: $userId, amount: $amount) {
                id
                userId
                amount
                user {
                    name
                    age
                }
            }
        }
        """
    )

    params = {
        "userId": user_id,
        "amount": amount
    }

    result = client.execute(add_transaction_mutation, variable_values=params)
    print(result)

# Example usage:
add_transaction("1", 100.50)

{'addTransaction': {'id': '3', 'userId': '1', 'amount': 100.5, 'user': {'name': 'Alice', 'age': 30}}}


/opt/homebrew/lib/python3.9/site-packages/graphql/language/ast.py:361: RuntimeWarning: coroutine 'main' was never awaited
  super().__setattr__(key, value)


In [39]:
add_transaction("3", 500.50)

{'addTransaction': {'id': '2', 'userId': '3', 'amount': 500.5, 'user': {'name': 'Charlie', 'age': 25}}}


You can navigate to the GraphQL playground - http://localhost:4000/graphql

and run the below 
<pre><code>
subscription {
  transactionAdded {
    id
    userId
    amount
    user {
      name
      age
    }
  }
}
</code></pre>

to see the subscriptions getting update for each add_transactions function calls